In [11]:
import pandas as pd
from openpyxl import load_workbook
import os
from openpyxl.utils import get_column_letter

no_retirement_config = [
    [8, 9, 10, 14, 15, 16], # columns I to K (8-10) and O to Q (14-16)
    20  # Start copying from row 20
]

with_retirement_config = [
    [8, 9, 10, 11, 17, 18, 19],  # Columns K to M and O to Q
    20  # Start copying from row 20
]

with_ltc_config = [
    [9, 10, 11, 16, 17, 18, 19],  # Columns J to L and Q to T
    20  # Start copying from row 20
]

def copy_paste_excel_contents(file_list_path, target_excel_path, sheet_name="DataSheet1", config='no_retirement'):
    # Open the target Excel workbook
    if os.path.exists(target_excel_path):
        target_wb = load_workbook(target_excel_path)
        # Check if the specified sheet exists, if not create it
        if sheet_name in target_wb.sheetnames:
            target_ws = target_wb[sheet_name]
        else:
            target_ws = target_wb.create_sheet(sheet_name)
    else:
        from openpyxl import Workbook
        target_wb = Workbook()
        target_ws = target_wb.create_sheet(sheet_name)
        target_wb.remove(target_wb['Sheet'])  # Remove the default created 'Sheet' if it exists

    if config  == 'no_retirement':
        columns_to_copy, copy_from_row = no_retirement_config
    elif config == 'with_retirement':
        columns_to_copy, copy_from_row = with_retirement_config
    elif config == 'with_ltc':
        columns_to_copy, copy_from_row = with_ltc_config

    num_columns_to_copy = len(columns_to_copy)

    col_offset = 0  # Start from the first column
    with open(file_list_path, 'r') as file:
        for filename in file:
            filename = filename.strip() + ".xlsx"

            try:
                # Read the Excel file into DataFrame, ensuring it has enough columns
                df = pd.read_excel(filename, skiprows=copy_from_row-2)  # Skip the first 19 rows to start at row 20
                if df.shape[1] < columns_to_copy[-1] + 1:  # Check if there are at least 17 columns (up to 'Q')
                    print(f"Skipping {filename}, not enough columns.")
                    continue

                # Check if specific columns contain data from row 20 onwards
                if df.iloc[:, columns_to_copy].isnull().all().any():
                    print(f"Skipping {filename}, required columns contain only null values from row 20 onwards.")
                    continue

                # Select the columns and the required rows with actual content
                selected_data = df.iloc[:, columns_to_copy]
                num_rows_copied = selected_data.shape[0]

                # Calculate the target columns based on offset
                target_columns = [get_column_letter(col_offset + 1 + i) for i in range(num_columns_to_copy)]

                # Pasting the data into the target workbook
                for idx, col in enumerate(target_columns):
                    for row_idx, value in enumerate(selected_data.iloc[:, idx]):
                        cell_reference = f'{col}{14 + row_idx}'
                        target_ws[cell_reference].value = value

                # Update the column offset for the next file, add 7 instead of 6 to include one empty column
                col_offset += (num_columns_to_copy + 1)

                # Calculate the dynamic row for setting values
                #target_row = 14 + num_rows_copied - 26  # Calculate the dynamic row for setting values
                #specific_columns = ['D', 'K', 'R', 'Y']  # Specific columns for D11, K11, R11, Y11 adjustments

                # for i, col in enumerate(specific_columns):
                #     dynamic_cell = f'{get_column_letter(col_offset + i * 7)}{target_row}'
                #     target_ws[f'{col}11'].value = f"={dynamic_cell}"

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Save the target workbook
    target_wb.save(target_excel_path)
    target_wb.close()

# Usage example
#copy_paste_excel_contents('no_retirement_file_list.txt', 'target.xlsx', 'DataSheet')
copy_paste_excel_contents('with_retirement_file_list.txt', 'target.xlsx', 'DataSheet1', config='with_retirement')
copy_paste_excel_contents('with_ltc_file_list.txt', 'target.xlsx', "DataSheet2", config='with_ltc')
